# SCohenLab 2D BATCH Image Processing notebook (Simplified MCZ)

--------------
# PIPELINE OVERVIEW
## ❶ GOAL SETTING ✍

### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and SOMA) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_ ✅
- robust inference of subcellular objects:
  + 1️⃣-***nuclei***
  + 2️⃣-***soma***
  + 3️⃣-***cytosol*** (+ ***nucleus***)
  + 4️⃣-***lysosome***
  + 5️⃣-***mitochondria***
  + 6️⃣-***golgi***
  + 7️⃣-***peroxisome***
  + 8️⃣-***endoplasmic reticulum***
  + 9️⃣-***lipid body***





## ❷ DATA CREATION
> METHODS:📚📚
> 
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA 🏺 (artifacts)
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## ❸. IMAGE PROCESSING ⚙️🩻🔬
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, SOMA, CYTOSOL, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing 🌒
- Core-processing (thresholding) 🌕
- Post-processing  🌘

### QC 🚧 WIP 🚧 




## ❹. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A




## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union

import numpy as np
import pandas as pd
from skimage.measure import regionprops_table, regionprops, label
from skimage.morphology import binary_erosion


import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_czi_image,
                                                                    export_inferred_organelle,
                                                                    import_inferred_organelle,
                                                                    export_tiff,
                                                                    list_image_files)

from infer_subc_2d.utils.img import *
from infer_subc_2d.utils.stats import *

from infer_subc_2d.organelles import * 

import time
%load_ext autoreload
%autoreload 2



In [2]:
# NOTE:  these "constants" are only accurate for the testing MCZ dataset
from infer_subc_2d.constants import (TEST_IMG_N,
                                                                    NUC_CH ,
                                                                    LYSO_CH ,
                                                                    MITO_CH ,
                                                                    GOLGI_CH ,
                                                                    PEROXI_CH ,
                                                                    ER_CH ,
                                                                    LIPID_CH ,
                                                                    RESIDUAL_CH )              


## SETUP

In [3]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
in_data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

# save output ".tiff" files here
out_data_path = data_root_path / "out"

if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [4]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

source_file = meta_dict['file_name']

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:106: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [5]:
scale[0]/scale[1], scale

(7.267318290023735,
 (0.5804527163320905, 0.07987165184837318, 0.07987165184837318))

Now get the single "optimal" slice of all our organelle channels....

## get the inferred soma, nuclei and cytosol objects

(takes < 1 sec)

Builde the segmentations in order




In [6]:
###################
# SOMA, NUCLEI, CYTOSOL, NUCLEUS
###################
nuclei_obj =  get_nuclei(img_data,meta_dict, out_data_path)
soma_obj = get_soma(img_data, nuclei_obj, meta_dict, out_data_path)
cytosol_mask = get_cytosol(nuclei_obj , soma_obj , meta_dict, out_data_path)


>>>>>>>>>>>> tifffile.imread  (dtype=uint8in (0.03) sec
loaded  inferred 3D `nuclei`  from /Users/ahenrie/Projects/Imaging/data/out 
>>>>>>>>>>>> tifffile.imread  (dtype=int32in (0.03) sec
loaded  inferred 3D `soma`  from /Users/ahenrie/Projects/Imaging/data/out 
>>>>>>>>>>>> tifffile.imread  (dtype=uint8in (0.01) sec
loaded  inferred 3D `cytosol`  from /Users/ahenrie/Projects/Imaging/data/out 


-------------------------
## regionprops

`skimage.measure.regionprops` provides the basic tools nescessary to quantify our segmentations.

First lets see what works in 3D.  

> Note: the names of the regionprops correspond to the 2D analysis even for those which are well defined in 3D.  i.e. "area" is actually "volume" in 3D, etc.


-----------------
## basic stats

### per-organelle


- regionprops 


### summary stats

- group + aggregate:  surface_area, volume
  - median
  - mean
  - std 
  - count

- normalizers
  - SOMA?
  - CYTOSOL?

### nuclei caveats
The other organelles are sensibly normalized by cytosol.  does normalizing the nuclei by cytosol make sense?  or use soma?

Lets see which possible measures are sensible for 3D or volumetric with regionprops

In [29]:
labels = label(nuclei_obj )
rp = regionprops(labels, intensity_image=img_data[NUC_CH])

supported = [] 
unsupported = []

for prop in rp[0]:
    try:
        rp[0][prop]
        supported.append(prop)
    except NotImplementedError:
        unsupported.append(prop)

print("Supported properties:")
print("  " + "\n  ".join(supported))
print()
print("Unsupported properties:")
print("  " + "\n  ".join(unsupported))

Supported properties:
  area
  bbox
  bbox_area
  centroid
  convex_area
  convex_image
  coords
  equivalent_diameter
  euler_number
  extent
  feret_diameter_max
  filled_area
  filled_image
  image
  inertia_tensor
  inertia_tensor_eigvals
  intensity_image
  label
  local_centroid
  major_axis_length
  max_intensity
  mean_intensity
  min_intensity
  minor_axis_length
  moments
  moments_central
  moments_normalized
  slice
  solidity
  weighted_centroid
  weighted_local_centroid
  weighted_moments
  weighted_moments_central
  weighted_moments_normalized

Unsupported properties:
  eccentricity
  moments_hu
  orientation
  perimeter
  perimeter_crofton
  weighted_moments_hu


In [ ]:
# from scipy.ndimage import find_objects
    
# labels = label(nuclei_obj ).astype("int")
# objects = find_objects(labels)

# # objects are the slices into the original array for each organelle

In [7]:
# get overall summary stats for soma
florescence =  raw_soma_MCZ(img_data, scale_min_max=False)


In [9]:

def dump_stats(name: str, segmentation:np.ndarray, intensity_img:np.ndarray, mask:np.ndarray, out_data_path: Path, source_file: str) -> pd.DataFrame:
    """
    get summary stats
    calls `get_summary_stats_3D`
    """
    
    stats_table,_ = get_summary_stats_3D(segmentation, intensity_img, mask) 
    csv_path = out_data_path / f"{name}_{source_file.split('/')[-1].split('.')[0]}_stats.csv"
    stats_table.to_csv(csv_path )
    print(f"dumped {name} table to {csv_path}")

    return stats_table

    


In [10]:
soma_table = dump_stats("soma", soma_obj, florescence, soma_obj, out_data_path, source_file)
nucleus_table = dump_stats("nucleus", nuclei_obj, img_data[NUC_CH], soma_obj, out_data_path, source_file)

dumped soma table to /Users/ahenrie/Projects/Imaging/data/out/soma_ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed_stats.csv
dumped nucleus table to /Users/ahenrie/Projects/Imaging/data/out/nucleus_ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed_stats.csv


In [11]:
soma_table.head()

,label,max_intensity,mean_intensity,min_intensity,volume,equivalent_diameter,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5,euler_number,extent,standard_deviation_intensity,surface_area
0,1,397989,18123.944078,0,1493502,141.819428,5.477527,377.567316,298.462994,0,65,84,12,626,636,1,0.401904,27336.770848,247666.953125


Now we want to get a list of our organelle names, segmentations, intensities (florescence)

In [12]:
# names of organelles we have
organelle_names = ["lysosome", "mitochondria","golgi","peroxisome","er","lipid"]

get_methods  = [get_lysosome,
            get_mitochondria,
            get_golgi,
            get_peroxisome,
            get_endoplasmic_reticulum,
            get_lipid]

# load all the organelle segmentations
organelles = [meth(img_data,meta_dict, out_data_path) for meth in get_methods]

# get the intensities
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROXI_CH,ER_CH,LIPID_CH]

intensities = [img_data[ch] for ch in organelle_channels]


>>>>>>>>>>>> tifffile.imread  (dtype=uint8in (0.01) sec
loaded  inferred 3D `lysosome`  from /Users/ahenrie/Projects/Imaging/data/out 
loaded lysosome in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint8in (0.01) sec
loaded  inferred 3D `mitochondria`  from /Users/ahenrie/Projects/Imaging/data/out 
>>>>>>>>>>>> tifffile.imread  (dtype=uint8in (0.01) sec
loaded  inferred 3D `golgi`  from /Users/ahenrie/Projects/Imaging/data/out 
starting segmentation...
>>>>>>>>>>>> tifffile.imread  (dtype=uint8in (0.01) sec
loaded  inferred 3D `peroxisome`  from /Users/ahenrie/Projects/Imaging/data/out 
loaded peroxisome in (0.01) sec
>>>>>>>>>>>> tifffile.imread  (dtype=uint8in (0.01) sec
loaded  inferred 3D `er`  from /Users/ahenrie/Projects/Imaging/data/out 
>>>>>>>>>>>> tifffile.imread  (dtype=uint8in (0.01) sec
loaded  inferred 3D `lipid`  from /Users/ahenrie/Projects/Imaging/data/out 


In [63]:


# stats_tables = []
# props = []
# for org, ch in zip(organelles, organelle_channels):
#     stats_tab, prps = get_summary_stats_3D(org , img_data[ch] ,cytosol_mask )
#     stats_tables.append(stats_tab)
#     props.append(prps)
# ch_dict = dict(zip(organelle_names, organelle_channels))
# idx_ = dict(zip(organelle_names, range(len(organelle_names) )) )


-----------------
## CONTACTS (cross-stats)

### organelle cross stats


- regionprops 



- intersect for A vs all other organelles Bi
  - regionprops on A ∩ Bi

   
- contacts?
  - dilate then intersect?
  - loop through each sub-object for each 



In [13]:



def shell_cross_stats(organelle_names: List[str], organelles:List[np.ndarray], mask:np.ndarray, out_data_path: Path, source_file: str) -> int:
    """
    get all cross stats between organelles A and B, and "shell of A" and B.   "shell" is the boundary of A 
    calls `get_AintB_stats_3D`
    """
    count = 0
    for j,target in enumerate(organelle_names):    
        print(f"getting stats for A = {target}")
        A = organelles[j ]
        # loop over Bs
        for i,nmi in enumerate(organelle_names):    
            if  i  != j:
                # get overall stats of intersection
                print(f"  b = {nmi}")
                B = organelles[i]
                stats_tab = get_AintB_stats_3D(A,B, mask)    
                csv_path = out_data_path / f"{target}X{nmi}_{source_file.split('/')[-1].split('.')[0]}_stats.csv"
                stats_tab.to_csv(csv_path )

                e_stats_tab = get_AintB_stats_3D(A,B, cytosol_mask, erode_A=True)
                csv_path = out_data_path / f"{target}_shellX{nmi}_{source_file.split('/')[-1].split('.')[0]}_stats.csv"
                e_stats_tab.to_csv(csv_path )

                # add the list of touches 
                labB = label(B)
                count += 1
    return count

    


In [80]:

def organelle_stats(organelle_names: List[str], organelles:List[np.ndarray], intinsities:List[np.ndarray], mask:np.ndarray, out_data_path: Path, source_file: str) -> int:
    """
    get summary and all cross stats between organelles A and B

    calls `get_summary_stats_3D`
    """
    count = 0
    org_stats_tabs = []
    for j,target in enumerate(organelle_names):    
        print(f"getting stats for A = {target}")
        A = organelles[ j ]
        # A_stats_tab, rp = get_simple_stats_3D(A,mask)
        A_stats_tab, rp = get_summary_stats_3D(A , intinsities[ j ] ,mask )

        # loop over Bs
        for i,nmi in enumerate(organelle_names):    
            if  i != j:
                # get overall stats of intersection
                print(f"  b = {nmi}")
                B = organelles[i]

                count += 1 
                # add the list of touches 
                labB = label(B)

                ov = []
                B_labs = []
                labs = []
                for idx,lab in enumerate(A_stats_tab['label']): # loop over A_objects
                    xyz = tuple(rp[idx].coords.T)
                    cmp_org = labB[xyz]

                    #total number of overlapping pixels
                    overlap = sum(cmp_org>0)
                    # overlap?
                    b_labs = cmp_org[cmp_org>0]
                    b_js = np.unique(b_labs).tolist()

                    # if overlap > 0:
                    labs.append(lab)
                    ov.append(overlap)
                    B_labs.append(b_js)

                # add organelle B columns to A_stats_tab
                A_stats_tab[f"{nmi}_overlap"] = ov
                A_stats_tab[f"{nmi}_labels"] = B_labs  # might want to make this easier for parsing later

        # org_stats_tabs.append(A_stats_tab)
        csv_path = out_data_path / f"{target}_{source_file.split('/')[-1].split('.')[0]}_stats.csv"
        A_stats_tab.to_csv(csv_path)

        print(f"dumped {count} csvs")
    return count

    
# refactor to just to a target vs. list of probes
# for nuclei mask == soma
# for all oother mask == cytosol



In [14]:
source_file

'/Users/ahenrie/Projects/Imaging/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi'

In [15]:
n_files = organelle_stats(organelle_names, organelles,intensities, cytosol_mask, out_data_path, source_file)

NameError: name 'organelle_stats' is not defined

In [82]:
n_files = dump_cross_stats(organelle_names, organelles, cytosol_mask, out_data_path, source_file) 


getting stats for A = lysosome
  b = mitochondria
  b = golgi
  b = peroxisome
  b = er
  b = lipid
getting stats for A = mitochondria
  b = lysosome
  b = golgi
  b = peroxisome
  b = er
  b = lipid
getting stats for A = golgi
  b = lysosome
  b = mitochondria
  b = peroxisome
  b = er
  b = lipid
getting stats for A = peroxisome
  b = lysosome
  b = mitochondria
  b = golgi
  b = er
  b = lipid
getting stats for A = er
  b = lysosome
  b = mitochondria
  b = golgi
  b = peroxisome
  b = lipid
getting stats for A = lipid
  b = lysosome
  b = mitochondria
  b = golgi
  b = peroxisome
  b = er



-----------------
##  SUMMARY STATS  
> WARNING: (🚨🚨🚨🚨 WIP)
### normalizations.

- overlaps, normalized by CYTOSOL, A, and B
- per cell averages, medians, std, and totals

These is all pandas munging and very straightforward tabular manipulation.


In [16]:
target = organelle_names[1]

csv_path = out_data_path / f"{target}_{source_file.split('/')[-1].split('.')[0]}_stats.csv"

mito_table = pd.read_csv(csv_path)
mito_table.head()

,Unnamed: 0,label,max_intensity,mean_intensity,min_intensity,volume,equivalent_diameter,centroid-0,centroid-1,centroid-2,...,lysosome_overlap,lysosome_labels,golgi_overlap,golgi_labels,peroxisome_overlap,peroxisome_labels,er_overlap,er_labels,lipid_overlap,lipid_labels
0,0,1,11903,4167.276923,0,130,6.285139,0.469231,358.400000,236.900000,...,0,[],0,[],0,[],0,[],0,[]
1,1,2,30495,8793.699093,0,1213,13.231855,3.217642,473.165705,180.775763,...,0,[],0,[],0,[],0,[],0,[]
2,2,3,14685,4455.750000,0,180,7.005266,1.044444,531.055556,282.200000,...,0,[],0,[],0,[],0,[],0,[]
3,3,4,16996,6839.428000,0,250,7.815926,1.632000,263.692000,419.088000,...,13,[57],0,[],0,[],0,[],0,[]
4,4,5,20819,8595.902655,0,226,7.557357,1.628319,276.106195,443.203540,...,0,[],0,[],0,[],0,[],0,[]


In [19]:
mito_table.volume.mean()

867.0


-----------------
## DISTRIBUTION  
> WARNING: (🚨🚨🚨🚨 WIP)
### XY- summary
Segment image in 3D;
sum projection of binary image; 
create 5 concentric rings going from the edge of the nuclie to the edge of the soma (ideally these will be morphed to soma/nuclei shape as done in CellProfiler); 
measure intensity per ring (include nuclei as the center area to measure from)/ring area; 
the normalized measurement will act as a frequency distribution of that organelle starting from the nuclei bin going out to the cell membrane - 
Measurements needed: mean, median, and standard deviation of the frequency will be calculated

- pre-processing
  1. Make 2D sum projection of binary segmentation
  2. Create 5 concentric rings going out from the edge of the nuclei to the edge of the soma - these rings should be morphed to the shape of the nuclei and soma. 
  3. Use nucleus + concentric rings to mask the 2D sum project into radial distribution regions: nuclei = bin 1, ... largest/outter most ring = bin 6. See similar concept in CellProfiler: https://cellprofiler-manual.s3.amazonaws.com/CellProfiler-4.2.5/modules/measurement.html?highlight=distribution#module-cellprofiler.modules.measureobjectintensitydistribution"	
   
- per-object measurements
  - For each bin measure:
    1. pixel ""intensity""
    2. bin area"

- per-object calculations
  - per-object For each bin: 
  - sum of pixel intensity per bin / bin area"	

- per cell summary
  1. Create a frequency table with bin number of the x axis and normalized pixel intensity on the y-axis
  2. Measure the frequency distribution's mean, median, and standard deviation for each cell"




### Z- summary
Segment image in 3D;
measure area fraction of each organelle per Z slice;
these measurements will act as a frequency distribution of that organelle starting from the bottom of the soma (not including neurites) to the top of the soma;
measurements: mean, median, and standard deviation of the frequency distribution	

- pre-processing
  1. subtract nuclei from the soma --> soma cytosol
  2. mask organelle channels with soma cytosol mask

- per-object measurements
  - For each Z slice in the masked binary image measure:
    1. organelle area
    2. soma cytosol area

- per-object calculations
  - For each Z slice in the masked binary image: organelle area / soma cytosol area

- per cell summary
  1. create a frequency table with the z slice number on the x axis and the area fraction on the y axis
  2. Measure the frequency distribution's mean, median, and standard deviation for each cell"

In [ ]:

# get the flattened
flat_soma = soma_obj.sum(axis=0) >0
flat_nuclei = apply_mask(nuclei_obj,soma_obj).sum(axis=0)>0
d_to_edge = distance_to_edge( label(flat_soma) )


In [ ]:
colors = color_labels(label(flat_soma) + label(flat_nuclei))
d_to_edge = distance_to_edge( colors )


In [ ]:
viewer.close()
viewer =napari.view_image(d_to_edge)
viewer.add_image(flat_nuclei)
viewer.add_image(colors)


In [ ]:
Zflat_soma = soma_obj.sum(axis=(1,2)) >0
Zflat_nuclei = apply_mask(nuclei_obj,soma_obj).sum(axis=(1,2)) > 0

Zflat_soma

In [ ]:
cmp_org[0]=2
org_js = np.unique(cmp_org)
org_js

In [ ]:
# csv_path = out_data_path / f"{o}_{meta_dict["file_name"].split('/')[-1].split('.')[0]}_stats.csv"
meta_dict['file_name'].split('/')[-1].split('.')[0]

In [ ]:
from scipy.ndimage import maximum_position
from scipy.ndimage import sum as ndi_sum

from scipy.sparse import coo_matrix

from infer_subc_2d.utils.img import distance_to_edge, distance_transform_edt

def centers_of_labels(labels):
    """Return the i,j coordinates of the centers of a labels matrix
    
    The result returned is an 2 x n numpy array where n is the number
    of the label minus one, result[0,x] is the i coordinate of the center
    and result[x,1] is the j coordinate of the center.
    You can unpack the result as "i,j = centers_of_labels(labels)"
    """
    max_labels = np.max(labels)
    if max_labels == 0:
        return np.zeros((2, 0), int)

    result = scind.center_of_mass(
        np.ones(labels.shape), labels, np.arange(max_labels) + 1
    )
    result = np.array(result)
    if result.ndim == 1:
        result.shape = (2, 1)
        return result
    return result.transpose()


def maximum_position_of_labels(image, labels, indices):
    """Return the i,j coordinates of the maximum value within each object
    
    image - measure the maximum within this image
    labels - use the objects within this labels matrix
    indices - label #s to measure
    
    The result returned is an 2 x n numpy array where n is the number
    of the label minus one, result[0,x] is the i coordinate of the center
    and result[x,1] is the j coordinate of the center.
    """

    if len(indices) == 0:
        return np.zeros((2, 0), int)

    result = maximum_position(image, labels, indices)
    result = np.array(result, int)
    if result.ndim == 1:
        result.shape = (2, 1)
        return result
    return result.transpose()


def size_similarly(labels, secondary):
    """Size the secondary matrix similarly to the labels matrix

    labels - labels matrix
    secondary - a secondary image or labels matrix which might be of
                different size.
    Return the resized secondary matrix and a mask indicating what portion
    of the secondary matrix is bogus (manufactured values).

    Either the mask is all ones or the result is a copy, so you can
    modify the output within the unmasked region w/o destroying the original.
    """
    if labels.shape[:2] == secondary.shape[:2]:
        return secondary, np.ones(secondary.shape, bool)
    if labels.shape[0] <= secondary.shape[0] and labels.shape[1] <= secondary.shape[1]:
        if secondary.ndim == 2:
            return (
                secondary[: labels.shape[0], : labels.shape[1]],
                np.ones(labels.shape, bool),
            )
        else:
            return (
                secondary[: labels.shape[0], : labels.shape[1], :],
                np.ones(labels.shape, bool),
            )

    # Some portion of the secondary matrix does not cover the labels
    result = np.zeros(
        list(labels.shape) + list(secondary.shape[2:]), secondary.dtype
    )
    i_max = min(secondary.shape[0], labels.shape[0])
    j_max = min(secondary.shape[1], labels.shape[1])
    if secondary.ndim == 2:
        result[:i_max, :j_max] = secondary[:i_max, :j_max]
    else:
        result[:i_max, :j_max, :] = secondary[:i_max, :j_max, :]
    mask = np.zeros(labels.shape, bool)
    mask[:i_max, :j_max] = 1
    return result, mask


def fixup_scipy_ndimage_result(whatever_it_returned):
    """Convert a result from scipy.ndimage to a numpy array
    
    scipy.ndimage has the annoying habit of returning a single, bare
    value instead of an array if the indexes passed in are of length 1.
    For instance:
    scind.maximum(image, labels, [1]) returns a float
    but
    scind.maximum(image, labels, [1,2]) returns a list
    """
    if getattr(whatever_it_returned, "__getitem__", False):
        return np.array(whatever_it_returned)
    else:
        return np.array([whatever_it_returned])

def do_measurements(
        self,
        workspace,
        image_name,
        object_name,
        center_object_name,
        center_choice,
        bin_count_settings,
        dd,
    ):
    """Perform the radial measurements on the image set

    workspace - workspace that holds images / objects
    image_name - make measurements on this image
    object_name - make measurements on these objects
    center_object_name - use the centers of these related objects as
                    the centers for radial measurements. None to use the
                    objects themselves.
    center_choice - the user's center choice for this object:
                    C_SELF, C_CENTERS_OF_OBJECTS or C_EDGES_OF_OBJECTS.
    bin_count_settings - the bin count settings group
    dd - a dictionary for saving reusable partial results

    returns one statistics tuple per ring.
    """
    bin_count = bin_count_settings.bin_count.value
    wants_scaled = bin_count_settings.wants_scaled.value
    maximum_radius = bin_count_settings.maximum_radius.value
    image = workspace.image_set.get_image(image_name, must_be_grayscale=True)
    objects = workspace.object_set.get_objects(object_name)
    labels, pixel_data = crop_labels_and_image(objects.segmented, image.pixel_data)
    nobjects = np.max(objects.segmented)
    measurements = workspace.measurements


    d_to_edge = distance_to_edge(labels) # made a local version
    if center_object_name is not None:
        #
        # Use the center of the centering objects to assign a center
        # to each labeled pixel using propagation
        #
        center_objects = workspace.object_set.get_objects(center_object_name)

        center_labels, cmask = size_similarly(labels, center_objects.segmented)
        pixel_counts = fixup_scipy_ndimage_result(
            ndi_sum(
                np.ones(center_labels.shape),
                center_labels,
                np.arange(
                    1, np.max(center_labels) + 1, dtype=np.int32
                ),
            )
        )

        good = pixel_counts > 0
        i, j = ( centers_of_labels(center_labels) + 0.5).astype(int)
        ig = i[good]
        jg = j[good]
        lg = np.arange(1, len(i) + 1)[good]
        if center_choice == C_CENTERS_OF_OTHERS  # Reduce the propagation labels to the centers of the centering objects
            center_labels = np.zeros(center_labels.shape, int)
            center_labels[ig, jg] = lg

        cl, d_from_center = centrosome.propagate.propagate(  np.zeros(center_labels.shape), center_labels, labels != 0, 1)
        cl[labels == 0] = 0            # Erase the centers that fall outside of labels

        # If objects are hollow or crescent-shaped, there may be objects without center labels. As a backup, find the
        # center that is the closest to the center of mass.
        missing_mask = (labels != 0) & (cl == 0)
        missing_labels = np.unique(labels[missing_mask])

        if len(missing_labels):
            all_centers = centers_of_labels(labels)
            missing_i_centers, missing_j_centers = all_centers[
                :, missing_labels - 1
            ]
            di = missing_i_centers[:, np.newaxis] - ig[np.newaxis, :]
            dj = missing_j_centers[:, np.newaxis] - jg[np.newaxis, :]
            missing_best = lg[np.argsort(di * di + dj * dj)[:, 0]]
            best = np.zeros(np.max(labels) + 1, int)
            best[missing_labels] = missing_best
            cl[missing_mask] = best[labels[missing_mask]]

            # Now compute the crow-flies distance to the centers of these pixels from whatever center was assigned to the object.
            iii, jjj = np.mgrid[0 : labels.shape[0], 0 : labels.shape[1]]
            di = iii[missing_mask] - i[cl[missing_mask] - 1]
            dj = jjj[missing_mask] - j[cl[missing_mask] - 1]
            d_from_center[missing_mask] = np.sqrt(di * di + dj * dj)
    else:
        # Find the point in each object farthest away from the edge.
        # This does better than the centroid:
        # * The center is within the object
        # * The center tends to be an interesting point, like the center of the nucleus or the center of one or the other of two touching cells.
        i, j = maximum_position_of_labels(   d_to_edge, labels, objects.indices )
        center_labels = np.zeros(labels.shape, int)
        center_labels[i, j] = labels[i, j]
        # Use the coloring trick here to process touching objectsin separate operations
        colors = color_labels(labels)
        ncolors = np.max(colors)
        d_from_center = np.zeros(labels.shape)
        cl = np.zeros(labels.shape, int)

        for color in range(1, ncolors + 1):
            mask = colors == color
            l, d = centrosome.propagate.propagate( np.zeros(center_labels.shape), center_labels, mask, 1)
            d_from_center[mask] = d[mask]
            cl[mask] = l[mask]
        good_mask = cl > 0

        if center_choice == C_EDGES_OF_OTHER:
            # Exclude pixels within the centering objects
            # when performing calculations from the centers
            good_mask = good_mask & (center_labels == 0)
        i_center = np.zeros(cl.shape)
        i_center[good_mask] = i[cl[good_mask] - 1]
        j_center = np.zeros(cl.shape)
        j_center[good_mask] = j[cl[good_mask] - 1]
        normalized_distance = np.zeros(labels.shape)

        if wants_scaled:
            total_distance = d_from_center + d_to_edge
            normalized_distance[good_mask] = d_from_center[good_mask] / ( total_distance[good_mask] + 0.001 )
        else:
            normalized_distance[good_mask] = (d_from_center[good_mask] / maximum_radius)
        dd[name] = [normalized_distance, i_center, j_center, good_mask]



    ngood_pixels = np.sum(good_mask)
    good_labels = labels[good_mask]
    bin_indexes = (normalized_distance * bin_count).astype(int)
    bin_indexes[bin_indexes > bin_count] = bin_count
    labels_and_bins = (good_labels - 1, bin_indexes[good_mask])

    histogram = coo_matrix( (pixel_data[good_mask], labels_and_bins), (nobjects, bin_count + 1) ).toarray()

    sum_by_object = np.sum(histogram, 1)
    sum_by_object_per_bin = np.dstack([sum_by_object] * (bin_count + 1))[0]
    fraction_at_distance = histogram / sum_by_object_per_bin
    number_at_distance = coo_matrix)(np.ones(ngood_pixels), labels_and_bins), (nobjects, bin_count + 1)).toarray()

    object_mask = number_at_distance > 0
    sum_by_object = np.sum(number_at_distance, 1)
    sum_by_object_per_bin = np.dstack([sum_by_object] * (bin_count + 1))[0]
    fraction_at_bin = number_at_distance / sum_by_object_per_bin
    mean_pixel_fraction = fraction_at_distance / ( fraction_at_bin + np.finfo(float).eps    )
    masked_fraction_at_distance = np.ma.masked_array( fraction_at_distance, ~object_mask )
    masked_mean_pixel_fraction = np.ma.masked_array(mean_pixel_fraction, ~object_mask)

    # Anisotropy calculation.  Split each cell into eight wedges, then compute coefficient of variation of the wedges' mean intensities
    # in each ring. Compute each pixel's delta from the center object's centroid
    i, j = np.mgrid[0 : labels.shape[0], 0 : labels.shape[1]]
    imask = i[good_mask] > i_center[good_mask]
    jmask = j[good_mask] > j_center[good_mask]
    absmask = abs(i[good_mask] - i_center[good_mask]) > abs(
        j[good_mask] - j_center[good_mask]
    )
    radial_index = (
        imask.astype(int) + jmask.astype(int) * 2 + absmask.astype(int) * 4
    )
    statistics = []

    for bin in range(bin_count + (0 if wants_scaled else 1)):
        bin_mask = good_mask & (bin_indexes == bin)
        bin_pixels = np.sum(bin_mask)
        bin_labels = labels[bin_mask]
        bin_radial_index = radial_index[bin_indexes[good_mask] == bin]
        labels_and_radii = (bin_labels - 1, bin_radial_index)
        radial_values = coo_matrix( (pixel_data[bin_mask], labels_and_radii), (nobjects, 8) ).toarray()
        pixel_count = coo_matrix( (np.ones(bin_pixels), labels_and_radii), (nobjects, 8) ).toarray()

        mask = pixel_count == 0
        radial_means = np.ma.masked_array(radial_values / pixel_count, mask)
        radial_cv = np.std(radial_means, 1) / np.mean(radial_means, 1)
        radial_cv[np.sum(~mask, 1) == 0] = 0

        for measurement, feature, overflow_feature in (
            (fraction_at_distance[:, bin], MF_FRAC_AT_D, OF_FRAC_AT_D),
            (mean_pixel_fraction[:, bin], MF_MEAN_FRAC, OF_MEAN_FRAC),
            (np.array(radial_cv), MF_RADIAL_CV, OF_RADIAL_CV),
        ):
            if bin == bin_count:
                measurement_name = overflow_feature % image_name
            else:
                measurement_name = feature % (image_name, bin + 1, bin_count)
            measurements.add_measurement(object_name, measurement_name, measurement)

            if feature in heatmaps:
                heatmaps[feature][bin_mask] = measurement[bin_labels - 1]

        radial_cv.mask = np.sum(~mask, 1) == 0
        bin_name = str(bin + 1) if bin < bin_count else "Overflow"

        statistics += [
            (
                image_name,
                object_name,
                bin_name,
                str(bin_count),
                np.round(np.mean(masked_fraction_at_distance[:, bin]), 4),
                np.round(np.mean(masked_mean_pixel_fraction[:, bin]), 4),
                np.round(np.mean(radial_cv), 4),
            )
        ]

    return statistics

In [ ]:
###################
# MITOCONDRIA
###################
mito_obj  = get_mitochondria(img_data,meta_dict, out_data_path)
mito_table = get_summary_stats_3D( mito_obj, img_data[MITO_CH],cytosol_mask)

###################
#  GOLGI
###################
golgi_obj = get_golgi(img_data,meta_dict, out_data_path)
golgi_obj = get_summary_stats_3D( golgi_obj, img_data[GOLGI_CH],cytosol_mask)

###################
#  PEROXISOME
###################
peroxisome_obj  = get_peroxisome(img_data,meta_dict, out_data_path)

###################
#  ER
###################
er_obj  = get_endoplasmic_reticulum(img_data,meta_dict, out_data_path)

###################
#  LIPID BODIES
###################
lipid_obj  =  get_lipid(img_data,meta_dict, out_data_path)

In [ ]:

viewer = napari.view_image(lysosome_obj)

In [ ]:
viewer.add_image(florescence)
viewer.add_image(cytosol_mask)


In [ ]:
from math import pi as PI

table['equivalent_diameter'], table['area'], (2 * 3 *  table['area'] / PI) ** (1 /3)


In [ ]:
###################
# SOMA, NUCLEI, CYTOSOL, NUCLEUS
###################
nuclei_obj =  infer_and_export_nuclei(img_data,meta_dict, out_data_path)

soma_obj = infer_and_export_soma(img_data, nuclei,meta_dict, out_data_path)
cytosol_mask =  infer_and_export_cytosol(soma_obj, nuclei_obj, meta_dict, out_data_path)



In [ ]:
viewer.add_image(cytosol_mask)


In [ ]:
###################
# LYSOSOME
###################
lysosome_obj  = get_lysosome(img_data,meta_dict, out_data_path)
florescence = apply_mask(img_data[LYSO_CH],cytosol_mask )  
lyso_table = get_summary_stats_3D(lysosome_obj, florescence)


In [ ]:

###################
# LYSOSOME
###################
lysosome = infer_and_export_lysosome(img_data,meta_dict, out_data_path)

###################
# MITOCONDRIA
###################
mitochondria = infer_and_export_mitochondria(img_data,meta_dict, out_data_path)
###################
#  GOLGI
###################
golgi = infer_and_export_golgi(img_data,meta_dict, out_data_path)

###################
#  PEROXISOME
###################
peroxisome = infer_and_export_peroxisome(img_data,meta_dict, out_data_path)

###################
#  ER
###################
er = infer_and_export_endoplasmic_reticulum(img_data,meta_dict, out_data_path)

###################
#  LIPID BODIES
###################
lipid =  infer_and_export_lipid(img_data,meta_dict, out_data_path)

In [ ]:
lipid

In [ ]:
from skimage.measure import regionprops_table

def collect_organelle_stats(
                                    img_in:np.ndarray, 
                                    florescence:Union[np.ndarray,None]=None
                                    ): 

    properties = ['label']
    extra_properties = []
    size = False 
    perimeter = False 
    shape = True 
    position = True 
    moments = True

    labels = label(img_in).astype("int").squeeze()
    intensity_image = None if florescence is None else florescence.squeeze()
    intensity = intensity_image is not None # depricate for now because the "infered object" doesn't have intensity


    if size:
        properties = properties + ['area', 'bbox_area',  'equivalent_diameter'] #'convex_area',

    if intensity:
        properties = properties + ['max_intensity', 'mean_intensity', 'min_intensity']
        # arguments must be in the specified order, matching regionprops
        def standard_deviation_intensity(region, intensities):
            return np.std(intensities[region])
        extra_properties.append(standard_deviation_intensity)

    if perimeter:
        if len(labels.shape) == 2:
            properties = properties + ['perimeter', 'perimeter_crofton']
        else:
            print("Perimeter measurements are not supported in 3D")
            # warnings.warn("Perimeter measurements are not supported in 3D")

    if shape:
        properties = properties + ['solidity', 'extent', 'feret_diameter_max', 'local_centroid']
        if len(labels.shape) == 2:
            properties = properties + ['major_axis_length', 'minor_axis_length', 'orientation', 'eccentricity']
            # we need these two to compute some shape descriptors
            if not size:
                properties = properties + ['area']
            if not perimeter:
                properties = properties + ['perimeter']
        else:
            properties = properties + ['moments_central']
        # euler_number,

    if position:
        properties = properties + ['centroid', 'bbox', 'weighted_centroid']

    if moments:
        properties = properties + ['moments', 'moments_normalized']
        if 'moments_central' not in properties:
            properties = properties + ['moments_central']
        if len(labels.shape) == 2:
            properties = properties + ['moments_hu']

    # todo:
    # weighted_local_centroid
    # weighted_moments
    # weighted_moments_central
    # weighted_moments_hu
    # weighted_moments_normalized

    # quantitative analysis using scikit-image's regionprops
    print(labels.shape)
    print(properties)
    table = regionprops_table(labels, intensity_image=intensity_image,
                                properties=properties, extra_properties=extra_properties)
    return table


In [ ]:




organelles = [nuclei,
                            lysosome,
                            mitochondria,
                            golgi,
                            peroxisome,
                            er,
                            lipid]

organelle_names = ["nuclei","lysosome", "mitochondria","golgi","peroxisome","er","lipid"]
organelle_channel = [NUC_CH,LYSO_CH,MITO_CH,GOLGI_CH,PEROXI_CH,ER_CH,LIPID_CH]

stats_bag = []
for i,o in enumerate(organelle_names):
    print(f"{o}- {organelle_channel[i]}")
    org = organelles[i]
    ch = organelle_channel[i]
    florescence = img_data[ch]

    # apply mask 
    mask = soma if ch==NUC_CH else cytosol

    org = apply_mask(org, mask)
    florescence = apply_mask(org,mask)  
    _stats = collect_organelle_stats(org, florescence)
    stats_bag.append(_stats)

    # get lablels of masked organelle

    # export



In [ ]:
stats_bag = []
for i,o in enumerate(organelle_names):
    print(f"{o}- {organelle_channel[i]}")
    org = organelles[i]
    ch = organelle_channel[i]
    florescence = img_data[ch]

    # apply mask 
    mask = soma if ch==NUC_CH else cytosol

    org = apply_mask(org, mask)
    florescence = apply_mask(org,mask)  
    _stats = collect_organelle_stats(org, florescence)
    stats_bag.append(_stats)

    # get lablels of masked organelle

    # export


    

In [ ]:

###################
# LYSOSOME
###################
lysosome_object =  fixed_infer_lysosome(img_data) 

###################
# MITOCONDRIA
###################
mito_object =  fixed_infer_mitochondria(img_data) 
###################
#  GOLGI
###################
golgi_object =  fixed_infer_golgi(img_data) 

###################
#  PEROXISOME
###################
peroxi_object =  fixed_infer_peroxisome(img_data) 

###################
#  ER
###################
er_object =  fixed_infer_endoplasmic_reticulum(img_data) 

###################
#  LIPID BODIES
###################
lipid_object =  fixed_infer_lipid(img_data) 


In [ ]:
img_layers = [soma_mask,
                            nuclei_object,
                            cytosol_mask,
                            lysosome_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            lipid_object ]


organelle_names = ["soma","nuclei","cytosol","lysosome", "mitochondria","golgi","peroxisome"]

In [ ]:
labels = label(soma_mask).astype("int")


In [ ]:
regionprops_table(labels)

In [ ]:
# import pandas as pd
# # load region properties from csv file
# reg_props = pd.read_csv(csv_filename)
# try:
#     edited_reg_props = reg_props.drop(["Unnamed: 0"], axis=1)
# except KeyError:
#     edited_reg_props = reg_props

# if "label" not in edited_reg_props.keys().tolist():
#     label_column = pd.DataFrame(
#         {"label": np.array(range(1, (len(edited_reg_props) + 1)))}
#     )
#     edited_reg_props = pd.concat([label_column, edited_reg_props], axis=1)

In [ ]:
def handle_organelles(
        segmentation:Union[np.ndarray,None]=None, ,
        florescence:Union[np.ndarray,None]=None, 
        organelle: int,
        mask:Union[np.ndarray,None]=None, 
        # infer_params: Union[dict, None]=None,
        # prior:Union[Any,None]=None, 
        # export_nm:Union[Path, str,None]=None, 
        ) -> np.ndarray:
    """
    Procedure to get 
    
    Params:
    ----------
    segmentation:
        np.ndarray containing inferred organelle object

    florescence:
        np.ndarray containing raw organelle image

    organelle:
        channel of organelle to process: nuclei,  NUC_CH = 0, LYSO_CH = 1, MITO_CH = 2, GOLGI_CH = 3, PEROXI_CH = 4, ER_CH = 5, LIPID_CH = 6, RESIDUAL_CH = 7

    in_image:
        optional np.ndarray containing raw organelle image

    mask:
        optional mask (soma or cytosol)
    
    stats:
        optiohal Prior class
    
    export:
        boolean flag to export - default True
    """
    # CHOOSE which inferred organelle to 
    if organelle == NUC_CH: #0
        organelle_name = 'nucleus'
    elif organelle == LYSO_CH: #1 
        organelle_name = 'lysosome'
    elif organelle == MITO_CH: #2
        organelle_name = 'mitochondria'
    elif organelle == GOLGI_CH: #3
        organelle_name = 'golgi'
    elif organelle == PEROXI_CH: #4
        organelle_name = 'peroxisome'
    elif organelle == ER_CH: #5
        organelle_name = 'endoplasmic reticulum'
    elif organelle == LIPID_CH: #6 
        organelle_name = 'lipid'
    elif organelle == RESIDUAL_CH: #7
        organelle_name = 'residual'

    print(organelle_name)

    target = segmentation[organelle].squeeze()
    target = apply_mask(target, mask.squeeze())    
    target_intensity = florescence[organelle].squeeze()

    summary_stats = collect_organelle_stats(
                                    target, 
                                    target_intensity
                                    )
    # test if it worked
    labels = label(img_in).astype("int").squeeze()

    # export

    # return


In [ ]:
properties = ['label']
extra_properties = []
size = True 
perimeter = True 
shape = True 
position = True 
moments = True

labels = label(img_in).astype("int").squeeze()
intensity_image = None if florescence is None else florescence.squeeze()
intensity = intensity_image is not None # depricate for now because the "infered object" doesn't have intensity


if size:
    properties = properties + ['area', 'bbox_area',  'equivalent_diameter'] #'convex_area',

if intensity:
    properties = properties + ['max_intensity', 'mean_intensity', 'min_intensity']
    # arguments must be in the specified order, matching regionprops
    def standard_deviation_intensity(region, intensities):
        return np.std(intensities[region])
    extra_properties.append(standard_deviation_intensity)

if perimeter:
    if len(labels.shape) == 2:
        properties = properties + ['perimeter', 'perimeter_crofton']
    else:
        print("Perimeter measurements are not supported in 3D")
        # warnings.warn("Perimeter measurements are not supported in 3D")
        properties = properties + ['perimeter', 'perimeter_crofton']

if shape:
    properties = properties + ['solidity', 'extent', 'feret_diameter_max', 'local_centroid']
    if len(labels.shape) == 2:
        properties = properties + ['major_axis_length', 'minor_axis_length', 'orientation', 'eccentricity']
        # we need these two to compute some shape descriptors
        if not size:
            properties = properties + ['area']
        if not perimeter:
            properties = properties + ['perimeter']
    else:
        properties = properties + ['moments_central']
    # euler_number,

if position:
    properties = properties + ['centroid', 'bbox', 'weighted_centroid']

if moments:
    properties = properties + ['moments', 'moments_normalized']
    if 'moments_central' not in properties:
        properties = properties + ['moments_central']
    if len(labels.shape) == 2:
        properties = properties + ['moments_hu']

# todo:
# weighted_local_centroid
# weighted_moments
# weighted_moments_central
# weighted_moments_hu
# weighted_moments_normalized

# quantitative analysis using scikit-image's regionprops
print(labels.shape)
print(properties)
table = regionprops_table(labels, intensity_image=intensity_image,
                            properties=properties, extra_properties=extra_properties)

In [ ]:
organelle = 1
mask = cytosol_mask
segmentation = np.stack([
                            nuclei_object,
                            lysosome_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            lipid_object ])

florescence = img_2D[:-1] # throw out residual


florescence.shape, segmentation.shape, mask.shape


In [ ]:

# CHOOSE which inferred organelle to 
if organelle == NUC_CH: #0
    organelle_name = 'nucleus'
elif organelle == LYSO_CH: #1 
    organelle_name = 'lysosome'
elif organelle == MITO_CH: #2
    organelle_name = 'mitochondria'
elif organelle == GOLGI_CH: #3
    organelle_name = 'golgi'
elif organelle == PEROXI_CH: #4
    organelle_name = 'peroxisome'
elif organelle == ER_CH: #5
    organelle_name = 'endoplasmic reticulum'
elif organelle == LIPID_CH: #6 
    organelle_name = 'lipid'
elif organelle == RESIDUAL_CH: #7
    organelle_name = 'residual'

print(organelle_name)

target = segmentation[organelle].squeeze()
target = apply_mask(target, mask.squeeze())    
target_intensity = florescence[organelle].squeeze()

labels = label(target).astype("int").squeeze()


summary_stats = collect_organelle_stats(
                                target, 
                                target_intensity
                                )
# target.shape, target_intensity.shape, labels.shape

labels.max()



In [ ]:

# loop over all labeled organelles
for org_i in range(1,labels.max()+1):
    # extract org_i mask
    msk_i = labels == org_i
    intersect = 


In [ ]:
from skimage.measure import regionprops

label_image = labels

input_props = regionprops(
    label_image, intensity_image=None, cache=True
)


In [ ]:

input_props[0].coords.shape

In [ ]:

input_centroids = [np.int_(obj["centroid"]) for obj in input_props]
input_centroids[0]


In [ ]:

output_segmented = np.zeros_like(label_image)

for ind, arr in enumerate(input_centroids):
    output_segmented[tuple(arr)] = ind + 1
    break



In [ ]:
ind

In [ ]:
napari.view_image(output_segmented)

In [ ]:

summary_stats = collect_organelle_stats(
                                target, 
                                target_intensity
                                )


import pandas as pd

stats_table = pd.DataFrame(summary_stats)
stats_table.head()

In [ ]:
# np.stack(img_layers, axis=0).shape
nuclei_table =  collect_organelle_stats(nuclei_object)


import pandas as pd

nuc = pd.DataFrame(nuclei_table)


In [ ]:
nuc

In [ ]:
lysosome_table =  collect_organelle_stats(lysosome_object)


In [ ]:
napari.view_image(lysosome_object)

In [ ]:
napari.view_labels(label(lysosome_object))

In [ ]:
lysosome_table

## Visualize with `napari` 1
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(
    nuclei_object,
    scale=scale,
    colormap='blue', 
    blending='additive'
)
viewer.add_image(
    lysosome_object,
    scale=scale,
    colormap='cyan', 
    blending='additive'
)

viewer.add_image(
    mito_object,
    scale=scale,
    colormap='green', 
    blending='additive'
)

viewer.add_image(
    golgi_object,
    scale=scale,
    colormap='yellow', 
    blending='additive'
)


viewer.add_image(
    peroxi_object,
    scale=scale,
    colormap='bop orange', 
    blending='additive'
)


viewer.add_image(
    er_object,
    scale=scale,
    blending='additive')

viewer.add_image(
    lipid_object,
    scale=scale,
    blending='additive')



In [ ]:
viewer.scale_bar.visible = True
from napari.utils.notebook_display import nbscreenshot

# viewer.dims.ndisplay = 3
# viewer.camera.angles = (-30, 25, 120)
nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()


There may be a bug where the input images to the "infer_*" functions are modified in place and we might need to access them.  _MASKING_ seems to be the problem.  Also need to be clear about _when_ to apply the mask.

In [ ]:
from infer_subc_2d.utils.img import select_z_from_raw
###########
# infer organelles
##########
def _fixed_infer_organelles(img_data):
    """
    wrapper to infer all organelles from a single multi-channel image
    """
    # ch_to_agg = (LYSO_CH, MITO_CH, GOLGI_CH, PEROXI_CH, ER_CH, LIPID_CH)

    # nuc_ch = NUC_CH
    # optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg)
    # # Stage 1:  nuclei, soma, cytosol
    # img_2D = select_z_from_raw(img_data, optimal_Z)
    img_2D = fixed_get_optimal_Z_image(img_data)

    soma_mask = fixed_infer_soma(img_2D)

    nuclei_object = fixed_infer_nuclei(img_2D, soma_mask)

    cytosol_mask = infer_cytosol(nuclei_object, soma_mask)

    # cyto masked objects.
    lysosome_object = fixed_infer_lysosome(img_2D, cytosol_mask)
    mito_object = fixed_infer_mitochondria(img_2D, cytosol_mask)
    golgi_object = fixed_infer_golgi(img_2D, cytosol_mask)
    peroxi_object = fixed_infer_peroxisome(img_2D, cytosol_mask)
    er_object = fixed_infer_endoplasmic_reticulum(img_2D, cytosol_mask)
    lipid_object = fixed_infer_lipid(img_2D, cytosol_mask)

    img_layers = [
        nuclei_object,
        lysosome_object,
        mito_object,
        golgi_object,
        peroxi_object,
        er_object,
        lipid_object,
        soma_mask,
        cytosol_mask,
    ]

    layer_names = [
        "nuclei",
        "lysosome",
        "mitochondria",
        "golgi",
        "peroxisome",
        "er",
        "lipid_body",
        "soma_mask",
        "cytosol_mask",
    ]
    # TODO: pack outputs into something napari readable
    img_out = np.stack(img_layers, axis=0)
    return (img_out, layer_names, optimal_Z)


In [ ]:

def stack_organelle_objects(soma_mask,
                            nuclei_object,
                            cytosol_mask,
                            lysosome_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            lipid_object) -> np.ndarray:
    """ wrapper to stack the inferred objects into a single numpy.ndimage """
    img_layers = [soma_mask,
                            nuclei_object,
                            cytosol_mask,
                            lysosome_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            lipid_object]
    return np.stack(img_layers, axis=0) 

In [ ]:
def stack_organelle_layers(*layers) -> np.ndarray:
    """ wrapper to stack the inferred objects into a single numpy.ndimage """

    return np.stack(layers, axis=0) 

In [ ]:
stacked_organelle_objects(*img_layers).shape

In [ ]:

inferred_organelles, layer_names, optimal_Z = _fixed_infer_organelles(img_data)


In [ ]:
viewer = napari.Viewer()


In [ ]:
cmaps = ['blue','cyan','green','yellow','bop orange','magenta','gray','gray','gray']

for i,organelle in enumerate(inferred_organelles):
    viewer.add_image(
        organelle,
        scale=scale,
        blending='additive',
        name = layer_names[i],
        colormap=cmaps[i]
    )


In [ ]:
viewer.scale_bar.visible = True

nbscreenshot(viewer, canvas_only=True)

In [ ]:
meta_dict

In [ ]:

##################
# export
##################

def _export_infer_organelles(img_out, layer_names, meta_dict, data_root_path):
       # get some top-level info about the RAW data
    # channel_names = meta_dict['name']
    # img = meta_dict['metadata']['aicsimage']
    # scale = meta_dict['scale']
    # channel_axis = meta_dict['channel_axis']
    img_name = meta_dict['file_name']
    # add params to metadata
    meta_dict['layer_names'] = layer_names
    out_path = data_root_path / "inferred_objects" 
    img_name_out = 'binarized_' + img_name.split("/")[-1].split(".")[0]

    out_file_n = export_ome_tiff(img_out, meta_dict, img_name_out, str(out_path)+"/", layer_names)
    print(f"saved file: {out_file_n}")
    return out_file_n

out_file_n = _export_infer_organelles(inferred_organelles, layer_names, meta_dict, data_root_path)



In [ ]:
out_file_n

----------------------

run a batch of ALL the images

First get all the images

In [ ]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"



now build a function to loop over them all and export




In [ ]:

def batch_process_all_czi(data_root_path):

    # linearly unmixed ".czi" files are here
    data_path = data_root_path / "raw"
    im_type = ".czi"
    # get the list of all files
    img_file_list = list_image_files(data_path,im_type)
    files_generated = []
    for czi_file in img_file_list:
        out_fn = process_czi_image(czi_file)
        files_generated.append(out_fn)

    print(f"generated {len(files_generated)} ")
    return files_generated

def process_czi_image(czi_file_name):
    """wrapper for processing"""

    img_data,meta_dict = read_czi_image(czi_file_name)
    # # get some top-level info about the RAW data
    # channel_names = meta_dict['name']
    # img = meta_dict['metadata']['aicsimage']
    # scale = meta_dict['scale']
    # channel_axis = meta_dict['channel_axis']

    inferred_organelles, layer_names,optimal_Z = _infer_organelles(img_data)
    meta_dict['z_slice'] = optimal_Z
    out_file_n = _export_infer_organelles(inferred_organelles, layer_names, meta_dict, data_root_path)

    ## TODO:  collect stats... 

    return out_file_n

# chan_name = 'nuclei'
# out_path = data_root_path / "inferred_objects" 
# object_name = 'NU_object'

# NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
# NU_object = NU_bioim.image
# NU_labels = label(NU_object)

# # calculate a filter dimension for median filtering which considers the difference in scale of Z
# z_factor = scale[0]//scale[1]
# med_filter_size = 4 #2D 
# med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter

In [ ]:
output_tiffs = batch_process_all_czi(data_root_path)


Write the `infer_soma` spec to the widget json

In [ ]:
from infer_subc_2d.organelles_config.helper import add_function_spec_to_widget_json

_fixed_infer_organelles =  {
        "name": "infer all organelles (fixed parameters)",
        "python::module": "infer_subc_2d.batch.batch_process",
        "python::function": "fixed_infer_organelles",
        "parameters": None
        }

add_function_spec_to_widget_json("fixed_infer_organelles",_fixed_infer_organelles)

In [ ]:

_stack_organelle_objects =  {
        "name": "stack organelles, argv spelled out",
        "python::module": "infer_subc_2d.batch",
        "python::function": "stack_organelle_objects",
        "parameters": None
        }

add_function_spec_to_widget_json("stack_organelle_objects",_stack_organelle_objects)


_stack_organelle_layers =  {
        "name": "stack organelles, *argv ",
        "python::module": "infer_subc_2d.batch",
        "python::function": "stack_organelle_layers",
        "parameters": None
        }

add_function_spec_to_widget_json("stack_organelle_layers",_stack_organelle_layers)


-------------------------------
## Write workflow .json
Now that we've added our function specs we can compose workflows.

In [ ]:

def make_fixed_infer_organelles_batch_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # Stage 1:  nuclei, soma, cytosol
    ###################   
    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("core")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("core")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("core")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    ###################
    # Stage 2:  cyto masked objects
    ###################   
    step_name.append("5")
    function_name.append("fixed_infer_lysosome")
    category.append("core")
    parameter_values.append(None )
    parent.append([1,4])

    
    step_name.append("6")
    function_name.append("fixed_infer_mitochondria")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("7")
    function_name.append("fixed_infer_golgi")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("8")
    function_name.append("fixed_infer_peroxisome")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("9")
    function_name.append("fixed_infer_er") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("10")
    function_name.append("fixed_infer_lipid") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("11")
    function_name.append("stack_organelle_objects") 
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([2,3,4,5,6,7,8,9,10])


    # TODO: add export functions

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_fixed_infer_organelles_batch_dict = make_fixed_infer_organelles_batch_dict()

write_workflow_json("infer_fixed_infer_organelles_batch", infer_fixed_infer_organelles_batch_dict)

In [ ]:

def make_fixed_infer_organelles_batch_dict2():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw
    """
    
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # Stage 1:  nuclei, soma, cytosol
    ###################   
    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("core")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("core")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("core")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    ###################
    # Stage 2:  cyto masked objects
    ###################   
    step_name.append("5")
    function_name.append("fixed_infer_lysosome")
    category.append("core")
    parameter_values.append(None )
    parent.append([1,4])

    
    step_name.append("6")
    function_name.append("fixed_infer_mitochondria")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("7")
    function_name.append("fixed_infer_golgi")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("8")
    function_name.append("fixed_infer_peroxisome")
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("9")
    function_name.append("fixed_infer_er") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])

    step_name.append("10")
    function_name.append("fixed_infer_lipid") 
    category.append("core")
    parameter_values.append(None)
    parent.append([1,4])


    step_name.append("11")
    function_name.append("stack_organelle_objects") 
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([2,3,4,5,6,7,8,9,10])

    # TODO: add export functions

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_fixed_infer_organelles_batch_dict2 = make_fixed_infer_organelles_batch_dict2()

write_workflow_json("infer_fixed_infer_organelles_batch2", infer_fixed_infer_organelles_batch_dict2)

In [ ]:
#TODO: make infer_organelles function with the exhaustive list of parmaters
_infer_organelles =  {
        "name": "Infer Endoplasmic Reticulum",
        "python::module": "infer_subc_2d.organelles",
        "python::function": "infer_endoplasmic_reticulum",
        "parameters": {
                "filament_scale": {
                        "data_type": "float",
                        "increment": 0.05,
                        "max": 10,
                        "min": 0,
                        "widget_type": "slider"
                },
                "filament_cut": {
                        "data_type": "float",
                        "increment": 0.001,
                        "max": 0.5,
                        "min": 0,
                        "widget_type": "slider"
                },
                "small_obj_w": {
                        "data_type": "int",
                        "increment": 1,
                        "max": 50,
                        "min": 1,
                        "widget_type": "slider"
                }
        }
}

add_function_spec_to_widget_json("eeeinfer_organelles", _infer_organelles, overwrite=True )



35 files processed in 6 minutes 47 seconds!!

In [ ]:

tiff_img_data,tiff_meta_dict = read_czi_image(output_tiffs[-1])


In [ ]:
tiff_img_data.shape

img = tiff_meta_dict['metadata']['aicsimage']
img.dims

In [ ]:
from aicsimageio.writers import OmeTiffWriter
data_in = img_out
channel_names = [layer_names]
image_names = [img_name]
print(image_names)
# chan_names = meta_in['metadata']['aicsimage'].channel_names
dimension_order = ["CZYX"]

num_images = len(  [data_in.shape])
if data_in.dtype == "bool":
    data_in = data_in.astype(np.uint8)
    data_in[data_in > 0] = 255

physical_pixel_sizes = [meta_dict["metadata"]["aicsimage"].physical_pixel_sizes]
out_ome = OmeTiffWriter.build_ome(
        [data_in.shape],
        [data_in.dtype],
        channel_names=channel_names,  # type: ignore
        image_name=image_names,
        physical_pixel_sizes=physical_pixel_sizes,
        dimension_order=dimension_order,
    )